In [9]:
# 모듈 import

from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.utils import get_device
import wandb

In [11]:
wandb.init(project="jjh_test", 
entity = "level2-cv-10-detection",
name="Retina_0")

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


In [12]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
cfg = Config.fromfile('/data/ephemeral/home/level2-objectdetection-cv-10-1/baseline/mmdetection/configs/retinanet/retinanet_r18_fpn_1x_coco.py')
# cfg = Config.fromfile('./configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py')


root='../../dataset/'

# dataset config 수정
cfg.data.train.classes = classes
cfg.data.train.img_prefix = root
cfg.data.train.ann_file = root + 'train.json' # train json 정보
# cfg.train_pipeline[3]['policies'][0][0]['img_scale'] = (512,512) # Resize

cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json' # test json 정보
cfg.test_pipeline[1]['img_scale'] = (512,512) # Resize

cfg.data.samples_per_gpu = 4

cfg.seed = 2022
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/retina'

# cfg.model.roi_head.bbox_head.num_classes = 10
cfg.model.bbox_head.num_classes = 10


cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=3, interval=1)
cfg.device = get_device()

cfg.log_config.hooks = [
    dict(type='TextLoggerHook'),
    dict(type='MMDetWandbHook',
         init_kwargs={'project': 'MMDetection-tutorial'},
         interval=10,
         log_checkpoint=True,
         log_checkpoint_metadata=True,
         num_eval_images=100)]


In [13]:
# build_dataset
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=0.24s)
creating index...
index created!


In [5]:
# dataset 확인
datasets[0]


CocoDataset Train dataset with number of images 4883, and instance counts: 
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| category          | count | category      | count | category        | count | category    | count | category     | count |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| 0 [General trash] | 3965  | 1 [Paper]     | 6352  | 2 [Paper pack]  | 897   | 3 [Metal]   | 936   | 4 [Glass]    | 982   |
| 5 [Plastic]       | 2943  | 6 [Styrofoam] | 1263  | 7 [Plastic bag] | 5178  | 8 [Battery] | 159   | 9 [Clothing] | 468   |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+

In [14]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()

2024-01-11 14:00:33,718 - mmcv - INFO - initialize ResNet with init_cfg {'type': 'Pretrained', 'checkpoint': 'torchvision://resnet18'}
2024-01-11 14:00:33,719 - mmcv - INFO - load model from: torchvision://resnet18
2024-01-11 14:00:33,720 - mmcv - INFO - load checkpoint from torchvision path: torchvision://resnet18
2024-01-11 14:00:33,769 - mmcv - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

2024-01-11 14:00:33,776 - mmcv - INFO - initialize FPN with init_cfg {'type': 'Xavier', 'layer': 'Conv2d', 'distribution': 'uniform'}
2024-01-11 14:00:33,804 - mmcv - INFO - initialize RetinaHead with init_cfg {'type': 'Normal', 'layer': 'Conv2d', 'std': 0.01, 'override': {'type': 'Normal', 'name': 'retina_cls', 'std': 0.01, 'bias_prob': 0.01}}
2024-01-11 14:00:33,845 - mmcv - INFO - 
backbone.conv1.weight - torch.Size([64, 3, 7, 7]): 
PretrainedInit: load from torchvision://resnet18 
 
2024-01-11 14:00:33,846 - mmcv - INFO

In [15]:
# 모델 학습
train_detector(model, datasets[0], cfg, distributed=False, validate=False)
wandb.finish()

2024-01-11 14:00:36,479 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2024-01-11 14:00:36,481 - mmdet - INFO - Start running, host: root@instance-5484, work_dir: /data/ephemeral/home/level2-objectdetection-cv-10-1/develop/JJH/work_dirs/retina
2024-01-11 14:00:36,482 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) MMDetWandbHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) MMDetWandbHook                     
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook    

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


2024-01-11 14:01:06,048 - mmdet - WARNING - No meta information found in the runner. 
2024-01-11 14:01:06,049 - mmdet - WARNING - To log evaluation or checkpoint metadata in MMDetWandbHook, `EvalHook` or `DistEvalHook` in mmdet is required, please check whether the validation is enabled.
2024-01-11 14:01:16,325 - mmdet - INFO - Epoch [1][50/1221]	lr: 9.890e-04, eta: 0:49:59, time: 0.205, data_time: 0.055, memory: 1501, loss_cls: 1.1388, loss_bbox: 0.6738, loss: 1.8127, grad_norm: 0.5796
2024-01-11 14:01:24,278 - mmdet - INFO - Epoch [1][100/1221]	lr: 1.988e-03, eta: 0:44:12, time: 0.159, data_time: 0.009, memory: 1501, loss_cls: 1.1380, loss_bbox: 0.6552, loss: 1.7931, grad_norm: 0.7343
2024-01-11 14:01:32,275 - mmdet - INFO - Epoch [1][150/1221]	lr: 2.987e-03, eta: 0:42:15, time: 0.160, data_time: 0.008, memory: 1501, loss_cls: 1.1313, loss_bbox: 0.6388, loss: 1.7700, grad_norm: 1.0743
2024-01-11 14:01:40,232 - mmdet - INFO - Epoch [1][200/1221]	lr: 3.986e-03, eta: 0:41:09, time: 0.15

learning_rate,▃██████████████████████████▂▂▂▂▂▂▂▂▂▁▁▁▁
momentum,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/grad_norm,▄▁▅▆▇▇▇▇▇▇▇▆▇▆▆▆▇█▇█▇█▇█▇▇▇▇▆▆█▇▆▇█▆▆▆▆▇
train/loss,██▆▅▄▄▄▄▄▃▃▄▃▃▃▂▂▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▂▂▁▁▂▁▁▁
train/loss_bbox,███▇▅▄▄▄▄▄▄▄▃▄▃▃▃▃▃▃▃▂▂▂▃▃▂▂▂▂▂▁▂▂▂▂▂▂▂▁
train/loss_cls,██▅▅▄▃▃▃▄▃▃▃▂▃▂▂▂▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
learning_rate,0.0001
momentum,0.9
train/grad_norm,2.31708
train/loss,0.57263
train/loss_bbox,0.22395
